In [ ]:
import pandas as pd
from datetime import datetime

# CSV yükle
df = pd.read_csv("/home/yapayzeka/ahsen_bulbul/data/yargitay_cleaned_2025-08-21.csv")

# 1. extractedDates temizleme (tekrarları sil, standart formata çevir)
def clean_dates(date_str):
    if pd.isna(date_str):
        return None
    dates = date_str.split(",")
    cleaned = set()
    for d in dates:
        d = d.strip()
        try:
            # Gün.Ay.Yıl formatını ISO formatına çevir
            dt = datetime.strptime(d, "%d.%m.%Y")
            cleaned.add(dt.strftime("%Y-%m-%d"))
        except:
            continue
    return ",".join(sorted(cleaned))

df["extractedDates"] = df["extractedDates"].apply(clean_dates)

# 2. location düzeltme
df["location"] = df["location"].str.replace("Dairesisi", "Dairesi", regex=False)

# 3. esasNo ve kararNo parçalama
def split_no(value):
    if pd.isna(value):
        return None, None
    parts = value.split()
    if len(parts) == 2:
        return parts[0], parts[1].replace(".", "")
    return value, None

df[["esasNo_num", "esasNo_tip"]] = df["esasNo"].apply(lambda x: pd.Series(split_no(x)))
df[["kararNo_num", "kararNo_tip"]] = df["kararNo"].apply(lambda x: pd.Series(split_no(x)))

# Sonuç kaydet
df.to_csv("veriler_temiz.csv", index=False)
print("Temizlenmiş veriler veriler_temiz.csv dosyasına kaydedildi.")


In [ ]:
import pandas as pd
import re

# ------------------------
# 1. CSV Dosyasını Yükleme
# ------------------------
df = pd.read_csv("/home/yapayzeka/ahsen_bulbul/cleaning/veriler_temiz.csv")  
# path'i kendine göre ayarlayabilirsin

# ------------------------
# 2. Stopword listesi
# ------------------------
stopwords = {
    "ve", "veya", "ile", "ancak", "gibi", "olarak", "ise", "fakat", 
    "ama", "bir", "bu", "şu", "o", "de", "da", "ki", "ile", "veya",
    "iletişim", "üzerine", "sonra", "önce", "göre", "karar", "sayılı"
}

# ------------------------
# 3. Temizleme fonksiyonu
# ------------------------
def clean_raw_text(text: str) -> str:
    if not isinstance(text, str):
        return text
    
    text = text.replace("\n", " ").replace("\t", " ")
    text = text.replace('""', '"')
    text = re.sub(r'İçtihat Metni', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\s+', ' ', text).strip()
    
    words = text.split()
    words = [w for w in words if w.lower() not in stopwords]
    return " ".join(words)

# ------------------------
# 4. Fonksiyonu DataFrame'e uygula
# ------------------------
df["rawText_clean"] = df["rawText"].apply(clean_raw_text)

# ------------------------
# 5. Kontrol için ilk satırı yazdır
# ------------------------
print(df["rawText_clean"].iloc[0])


In [ ]:
import pandas as pd
import re

# ------------------------
# 1. CSV Dosyasını Yükleme
# ------------------------
df = pd.read_csv("/home/yapayzeka/ahsen_bulbul/cleaning/veriler_temiz.csv")

# ------------------------
# 2. Stopword listesi
# ------------------------
stopwords = {
    "ve", "veya", "ile", "ancak", "gibi", "olarak", "ise", "fakat", 
    "ama", "bir", "bu", "şu", "o", "de", "da", "ki", "ile", "veya",
    "iletişim", "üzerine", "sonra", "önce", "göre", "karar", "sayılı"
}

# ------------------------
# 3. Temizleme fonksiyonu
# ------------------------
def clean_raw_text(text: str) -> str:
    if not isinstance(text, str):
        return ""
    
    # Gereksiz boşluk ve semboller
    text = text.replace("\n", " ").replace("\t", " ")
    text = text.replace('""', '"')
    text = re.sub(r'İçtihat Metni', '', text, flags=re.IGNORECASE)
    text = re.sub(r'\s+', ' ', text).strip()
    
    # Stopword temizleme
    words = text.split()
    words = [w for w in words if w.lower() not in stopwords]
    return " ".join(words)

# ------------------------
# 4. Fonksiyonu DataFrame'e uygula
# ------------------------
df["rawText_clean"] = df["rawText"].apply(clean_raw_text)

# ------------------------
# 5. Yeni CSV'ye kaydet
# ------------------------
df.to_csv("last_temiz.csv", index=False)

print(f"Temizlenmiş veri kaydedildi")


In [ ]:
df = pd.read_csv("last_temiz.csv", sep=";", encoding="utf-8")

print(df.head())

In [ ]:
import pandas as pd

df = pd.read_csv("last_temiz.csv", sep=",", encoding="utf-8", on_bad_lines='skip')
print(df.head())


In [ ]:
import pandas as pd
import re

# CSV dosyasını oku
df = pd.read_csv("veriler_temiz.csv")

# Temizleme fonksiyonu
def clean_text(text):
    if pd.isna(text):
        return ""
    text = text.replace("\n", " ")         # Satır başlarını kaldır
    text = text.replace("\r", " ")         # Geri dönüş karakterleri varsa kaldır
    text = text.replace('"', '')           # Çift tırnakları kaldır
    text = re.sub(r"\.{2,}", "", text)     # Üç veya daha fazla nokta sil
    text = re.sub(r"\s+", " ", text)       # Fazla boşlukları tek boşluk yap
    text = text.strip()                     # Baş ve sondaki boşlukları kaldır
    return text

# rawText sütununu temizle
df["rawText_clean"] = df["rawText"].apply(clean_text)

# İlk 5 satırı göster
print(df[["rawText", "rawText_clean"]].head())

# İstersen temizlenmiş CSV olarak kaydedebilirsin
df.to_csv("son_temiz.csv", index=False)


In [1]:
import pandas as pd
import re

# CSV dosyasını oku
df = pd.read_csv("veriler_temiz.csv", sep=",", encoding="utf-8")

# Temizleme ve normalize etme fonksiyonu
def normalize_text(text):
    if pd.isna(text):
        return ""
    
    text = text.replace("\n", " ")           # Satır başlarını kaldır
    text = text.replace("\r", " ")           # Geri dönüş karakterleri kaldır
    text = text.replace('"', '')             # Çift tırnakları kaldır
    text = re.sub(r"\.{2,}", "", text)       # Üç veya daha fazla nokta sil
    text = re.sub(r"\s+", " ", text)         # Fazla boşlukları tek boşluk yap
    text = text.strip()                       # Baş ve sondaki boşlukları kaldır
    text = text.lower()                       # Tüm metni küçük harfe çevir
    return text

# rawText sütununu normalize et
df["rawText_normalized"] = df["rawText"].apply(normalize_text)

# Örnek görüntüle
print(df[["rawText", "rawText_normalized"]].head())

# İstersen CSV olarak kaydedebilirsin
df.to_csv("dosya_normalized.csv", index=False, encoding="utf-8")


                                             rawText  \
0  6. Hukuk Dairesi 2023/596 E. , 2024/257 K. \n ...   
1  6. Hukuk Dairesi 2022/3281 E. , 2024/117 K. \n...   
2  6. Hukuk Dairesi 2023/4462 E. , 2024/493 K. \n...   
3  6. Hukuk Dairesi 2023/4229 E. , 2024/136 K. \n...   
4  6. Hukuk Dairesi 2022/3993 E. , 2024/775 K. \n...   

                                  rawText_normalized  
0  6. hukuk dairesi 2023/596 e. , 2024/257 k. \n ...  
1  6. hukuk dairesi 2022/3281 e. , 2024/117 k. \n...  
2  6. hukuk dairesi 2023/4462 e. , 2024/493 k. \n...  
3  6. hukuk dairesi 2023/4229 e. , 2024/136 k. \n...  
4  6. hukuk dairesi 2022/3993 e. , 2024/775 k. \n...  


In [3]:
df = pd.read_csv("dosya_normalized.csv", encoding="utf-8", nrows=10)
print(df.head())


                        _id        location  \
0  6750524485b6640290c37b06  6.HukukDairesi   
1  6750524485b6640290c37b07  6.HukukDairesi   
2  6750524485b6640290c37b08  6.HukukDairesi   
3  6750524485b6640290c37b09  6.HukukDairesi   
4  6750524485b6640290c37b0a  6.HukukDairesi   

                                             rawText  \
0  6. Hukuk Dairesi 2023/596 E. , 2024/257 K. \n ...   
1  6. Hukuk Dairesi 2022/3281 E. , 2024/117 K. \n...   
2  6. Hukuk Dairesi 2023/4462 E. , 2024/493 K. \n...   
3  6. Hukuk Dairesi 2023/4229 E. , 2024/136 K. \n...   
4  6. Hukuk Dairesi 2022/3993 E. , 2024/775 K. \n...   

                                      extractedDates        esasNo  \
0                                         2024-01-18   2023/596 E.   
1  2009-06-26,2009-12-25,2009-12-31,2010-01-04,20...  2022/3281 E.   
2                              1992-04-10,2024-02-05  2023/4462 E.   
3                              2023-06-23,2024-01-15  2023/4229 E.   
4  2011-02-04,2011-02-07,2012-